# Concepts
There are several key concepts in `torchfsm`, which are key components to construct and solve a PDE. By properly using these components, we can utilize `torchfsm` in a very efficient way.

## Operator

`Operator` is the key concept in `torchfsm`. It represents a **spatial** operation of a variable. For example, the gradient operator is an operator that computes the gradient of a variable. The divergence operator is an operator that computes the divergence of a variable. The laplacian operator is an operator that computes the laplacian of a variable. 

In `torchfsm`, there are three types of operator classes: `LinearOperator`, `NonlinearOperator` and `Operator`. `LinearOperator` class represents a linear terms in the context of Fourier Spectal method (See our [Theory Introduction](../theory/#linear-terms)). `NonlinearOperator` class represents a nonlinear terms in the context of Fourier Spectal method. `Operator` class is a combined operator that both contains linear and nonlinear terms.

All `Operator` class supports basic linear algebra operations, such as addition, subtraction and multiplication. Thanks to the linear feature of Fourier transformation, we can construct a PDE with different `Operator`s efficiently. For example, we can construct a [Burgers equation](https://en.wikipedia.org/wiki/Burgers%27_equation), $\frac{\partial \mathbf{u}}{\partial t}=-\mathbf{u} \cdot \nabla \mathbf{u} + \nu \nabla^2 \mathbf{u}$, as follows:

In [ ]:
from torchfsm.operator import Operator, Convection, Laplacian

def Burgers(nu:float) -> Operator:
    return nu*Laplacian()-Convection()
burgers=Burgers(0.01)

There are three important functions for `Operator` classes:

* `__call__`: Directly call the operator as a function will return the value of the operator applied to the input. This can be used to evaluate complex spatial derivatives.
* `integrate`: Integrate the operator in the time dimension. This is mainly used to solve time-dependent PDEs. 
* `solve`: Solve the linear operation with a given right-hand side. This only works for `LinearOperator` class and can be used to solve time-independent problems, such as Poisson equation.

In the later sections, we will further explain how to use these functions one by one.

All avaliable operators in `torchfsm` are summarized in the [cheet sheet](../cheetsheet/#operators).